In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from copy import deepcopy
from label_manipulation import median_filter_labels
import tensorflow as tf


In [2]:
from SignalRecognition.cwt_learner.wavelet_feature_engineering import CWT_learner
from signal_database.labelSelectedData import SignalDB,SignalBundle

In [4]:
import matplotlib.pyplot as plt
from plot_generator import plotResult_colorbars

In [6]:
sdb = SignalDB('grab_objects', path='./')
training_data_ = sdb.get_labeleddata()

In [7]:
%matplotlib

Using matplotlib backend: TkAgg


In [8]:
plt.figure()
for ii,example in enumerate(training_data_):
    plt.subplot(len(training_data_)*2,1,ii*2 + 1)
    for channel in example.signal_bundle.signals:
        plt.plot(channel)
    plt.subplot(len(training_data_)*2,1,ii*2 + 2)
    labels = example.labels
    plotResult_colorbars(labels, range(len(labels)))
plt.show()

In [19]:
all_possible_labels = []
for example in training_data_:
        all_possible_labels = all_possible_labels + example.labels
print set(all_possible_labels)



set(['', 'freespace', 'hold_foamblock', 'grab_hex', 'hand_on', 'grab_cup', 'release_foamblock', 'release_cup', 'hold_cup', 'grab_foamblock', 'still', 'hold_hex', 'release_hex', 'hand_off'])


In [9]:
training_data_relabeled = deepcopy(training_data_)


In [10]:
# label map 
label_map = {
    "freespace" : "freespace",
    "grab_hex" : "grab",
    "grab_cup" : "grab",
    "grab_foamblock" : "grab",
    "release_foamblock" : "release",
    "release_cup" : "release",
    "release_hex" : "release",
    "hold_cup" : "hold",
    "hold_hex" : "hold",
    "hold_foamblock" : "hold",
    "hand_on" : "hand_on",
    "hand_off" : "hand_off",
    "still" : "still",
    "" : ""
}



for example in training_data_relabeled:
        example.labels = [label_map[label] for label in example.labels]


In [12]:
cwt_learn = CWT_learner(signal_indices = [0,1,2,3])
training_data = training_data_relabeled[0:10]
testing_data = training_data_relabeled[10:15]
for ld in training_data:
    labels = [label for label in ld.labels]
    signals = []
    signals.append(list(np.array(ld.signal_bundle.signals[0]) +  np.array(ld.signal_bundle.signals[1]))) 
    signals.append(ld.signal_bundle.signals[2])
    
    signals.append(list(np.array(ld.signal_bundle.signals[3]) +  np.array(ld.signal_bundle.signals[4]))) 
    signals.append(ld.signal_bundle.signals[5])    
    
    cwt_learn.add_training_data(signals,labels)

In [13]:
all_labels = []
for example in training_data_relabeled:
        all_labels = example.labels + all_labels
label_types = list(set(all_labels))
print label_types

['', 'freespace', 'hold', 'hand_off', 'hand_on', 'release', 'grab', 'still']


In [47]:
# cwt_learn.train(classifier=SVC())
cwt_learn.train()
# cwt_learn.train(classifier=KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree'))

In [94]:
%matplotlib

Using matplotlib backend: TkAgg


In [48]:
fit_labels = []

for ii,ld in enumerate(testing_data):
    
    signals = []
    signals.append(list(np.array(ld.signal_bundle.signals[0]) +  np.array(ld.signal_bundle.signals[1]))) 
    signals.append(ld.signal_bundle.signals[2])
    signals.append(list(np.array(ld.signal_bundle.signals[3]) +  np.array(ld.signal_bundle.signals[4]))) 
    signals.append(ld.signal_bundle.signals[5])    
    
    labels = cwt_learn.fit(signals)
    fit_labels.append(labels)
    

In [54]:
plt.figure()
for ii,(ld,fit_label) in enumerate(zip(testing_data,fit_labels)):
    
    plt.subplot(len(testing_data)*2,1,ii*2 + 1)
    plotResult_colorbars(fit_label, range(len(fit_label)),labelNames=label_types)
    plt.subplot(len(testing_data)*2,1,ii*2 + 2)
    plotResult_colorbars(ld.labels, range(len(labels)),labelNames=label_types)
plt.show()

In [52]:
for ii,(ld,fit_label) in enumerate(zip(testing_data,fit_labels)):
    print len(ld.labels),len(fit_label)    



37855 37855
38059 38059
39893 39893
34573 34573
48552 48552


In [92]:
filtered_labels = median_filter_labels(fit_label,1001)
plt.subplot(2,1,1)
plotResult_colorbars(filtered_labels, range(len(labels)),labelNames=label_types)
plt.subplot(2,1,2)
plotResult_colorbars(ld.labels, range(len(labels)),labelNames=label_types)